In [20]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import pickle


In [3]:
df = pd.read_csv('/Users/aravindrajeshmenon/Documents/DataScienceProjects/Projects/Recommender-Model/data/spotify_data.csv')
df.head(5)


,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4


In [4]:
df = df.drop(['Unnamed: 0'], axis = 1)


In [5]:
df.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1159764 entries, 0 to 1159763
Data columns (total 19 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   artist_name       1159749 non-null  object 
 1   track_name        1159763 non-null  object 
 2   track_id          1159764 non-null  object 
 3   popularity        1159764 non-null  int64  
 4   year              1159764 non-null  int64  
 5   genre             1159764 non-null  object 
 6   danceability      1159764 non-null  float64
 7   energy            1159764 non-null  float64
 8   key               1159764 non-null  int64  
 9   loudness          1159764 non-null  float64
 10  mode              1159764 non-null  int64  
 11  speechiness       1159764 non-null  float64
 12  acousticness      1159764 non-null  float64
 13  instrumentalness  1159764 non-null  float64
 14  liveness          1159764 non-null  float64
 15  valence           1159764 non-null  float64
 16  

In [6]:
df.isna().sum()


artist_name         15
track_name           1
track_id             0
popularity           0
year                 0
genre                0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
duration_ms          0
time_signature       0
dtype: int64

Since this constitutes a miniscule amount of data points in the whole dataset, we can safely drop these data points. 

In [7]:
df = df.dropna()

In [8]:
df.describe()

,popularity,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,1.159748e+06,1.159748e+06,1.159748e+06,1.159748e+06,1.159748e+06,1.159748e+06,1.159748e+06,1.159748e+06,1.159748e+06,1.159748e+06,1.159748e+06,1.159748e+06,1.159748e+06,1.159748e+06,1.159748e+06
mean,1.838309e+01,2.011955e+03,5.374425e-01,6.396737e-01,5.287763e+00,-8.981292e+00,6.346560e-01,9.281520e-02,3.215380e-01,2.523423e-01,2.230197e-01,4.555689e-01,1.213775e+02,2.495587e+05,3.885880e+00
std,1.588563e+01,6.803908e+00,1.844753e-01,2.704993e-01,3.555204e+00,5.682203e+00,4.815267e-01,1.268416e-01,3.549873e-01,3.650701e-01,2.010712e-01,2.685169e-01,2.977964e+01,1.494243e+05,4.676983e-01
min,0.000000e+00,2.000000e+03,0.000000e+00,0.000000e+00,0.000000e+00,-5.810000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.073000e+03,0.000000e+00
25%,5.000000e+00,2.006000e+03,4.130000e-01,4.540000e-01,2.000000e+00,-1.082900e+01,0.000000e+00,3.710000e-02,6.400000e-03,1.050000e-06,9.790000e-02,2.260000e-01,9.879800e+01,1.810910e+05,4.000000e+00
50%,1.500000e+01,2.012000e+03,5.500000e-01,6.940000e-01,5.000000e+00,-7.450000e+00,1.000000e+00,5.070000e-02,1.470000e-01,1.760000e-03,1.340000e-01,4.380000e-01,1.219310e+02,2.257405e+05,4.000000e+00
75%,2.900000e+01,2.018000e+03,6.770000e-01,8.730000e-01,8.000000e+00,-5.276000e+00,1.000000e+00,8.900000e-02,6.400000e-01,6.140000e-01,2.920000e-01,6.740000e-01,1.399030e+02,2.869070e+05,4.000000e+00
max,1.000000e+02,2.023000e+03,9.930000e-01,1.000000e+00,1.100000e+01,6.172000e+00,1.000000e+00,9.710000e-01,9.960000e-01,1.000000e+00,1.000000e+00,1.000000e+00,2.499930e+02,6.000495e+06,5.000000e+00


In [10]:
df['artist_name'].nunique()

64158

In [11]:
df['genre'].value_counts()

genre
black-metal       21837
gospel            21621
ambient           21389
acoustic          21097
alt-rock          20918
                  ...  
chicago-house      5170
dubstep            4774
detroit-techno     3920
rock               3319
songwriter          589
Name: count, Length: 82, dtype: int64

In [12]:
df.columns

Index(['artist_name', 'track_name', 'track_id', 'popularity', 'year', 'genre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature'],
      dtype='object')

In [13]:
df['track_name'] = df['track_name'].str.lower()
scaler = StandardScaler()
features = ['popularity','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms']
df[features] = scaler.fit_transform(df[features])
df

,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Jason Mraz,i won't give up,53QF56cjZA9RTuuMZDrSA6,3.123384,2012,acoustic,-0.295121,-1.244639,-0.362219,-0.189488,0.758720,-0.393524,1.049226,-0.691216,-0.537222,-1.178954,0.403917,-0.062859,3
1,Jason Mraz,93 million miles,1s8tP3jP4GZcyHDsjvw218,1.990284,2012,acoustic,0.187328,-0.686411,-0.643497,-0.229613,0.758720,-0.528338,0.437937,-0.691179,-0.624753,0.221331,0.631455,-0.221997,4
2,Joshua Hyslop,do not let me go,7BRCa8MPiyuvr2VU3O9W0F,2.430934,2012,acoustic,-0.696259,-1.499723,-0.643497,-0.832373,0.758720,-0.477093,0.046373,-0.691079,-0.664042,-1.156609,0.619702,-0.606319,4
3,Boyce Avenue,fast car,63wsZUhUZLlh1OsyrZq7sz,2.493884,2012,acoustic,-0.788412,-1.436876,1.325448,-0.152002,0.758720,-0.445557,1.367548,-0.691216,-0.712781,0.195262,2.806734,0.366301,4
4,Andrew Belle,sky's still blue,6nXIYClvJAfi6ujLiKqEq8,2.242084,2012,acoustic,-0.582422,0.559434,0.200336,0.626921,-1.318008,-0.493649,-0.701259,-0.638350,-0.562088,-0.888469,1.695337,-0.035059,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159759,Nicola Conte,black spirits,0m27F0IGHLGAWhqd6ccYst,-0.905415,2011,trip-hop,-0.891407,0.378287,1.325448,0.444949,-1.318008,-0.151490,0.009752,-0.690830,-0.318394,0.247400,-0.450862,0.632121,3
1159760,Nicola Conte,quiet dawn,6er9p611eHEcUCU50j7D57,-0.968365,2011,trip-hop,-0.116235,0.130597,0.481614,0.245203,-1.318008,-0.474728,1.314025,-0.690863,-0.462621,-0.713434,-0.049715,0.237634,4
1159761,Amon Tobin,morning ms candis,7jsMMqxy1tt0rH5FzYcZTQ,-1.031315,2011,trip-hop,-0.251755,-0.738168,-0.080941,0.082590,0.758720,-0.515724,0.437937,-0.682643,-0.643652,-1.565894,-0.715304,-0.236278,4
1159762,Peace Orchestra,happy christmas (war is over),77lA1InUaXztuRk2vOzD1S,-1.157215,2011,trip-hop,-0.311383,-0.867558,-1.487331,-0.767609,0.758720,-0.514147,0.308355,-0.691043,-0.487488,-0.944332,0.420002,-0.069773,3


In [15]:
top_genres = df['genre'].value_counts().head(10).index
df['genre_clean'] = df['genre'].where(df['genre'].isin(top_genres), 'other')
genre_dummies = pd.get_dummies(df['genre_clean'], prefix='genre')
df = pd.concat([df, genre_dummies], axis=1)


In [16]:
df_small = df.sample(25000).reset_index(drop = True)

In [17]:
df_small.to_csv('../data/data_sample.csv')
